# Goal: Benchmark the CNNs based on HP opt done elsewhere

In [ ]:
target_elements_groups=[('Co','O'),('Fe','O'),('V','O'),('Cu','O'),
                        ('Ni','O'),('Cr','O'),('Mn','O'),('Ti','O')]
target_metals = set(['Co','Ni','Fe','Cr','V','Mn','Cu','Ti'])
target_elements_sets =[set(pair) for pair in target_elements_groups]
storage_directory = '/Users/steventorrisi/Documents/TRIXS/data/MP_OQMD_combined'


In [ ]:
import numpy as np
import pandas as pd
import os

# Machine learning:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D, \
 MaxPooling1D, Flatten
from keras import regularizers
from keras import optimizers
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tqdm import tqdm_notebook
# Plotting
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

import json
from trixs.machine_learning.benchmarks import plot_coordination_confusion_matrix
from trixs.machine_learning.util import onehot_reverse

In [ ]:
def classification_model(x_train, y_train, x_val, y_val, params):

    # Params is the vessel that will carry the optimzation parameters
    dropout = params['dropout']
    act = params['activation_function']
    optimizer = params['optimizer']

    #layers = params['layers']

    model = Sequential()

    # CNN vs. MLP.
    if params['cnn']:
        x_train = np.expand_dims(x_train, axis=-1)
        x_val = np.expand_dims(x_val, axis=-1)
        model.add(Conv1D(params['kernel'],
                         params['n_filters'],
                         strides=params['strides'], padding='valid',
                         activation=act,
                         input_shape=(x_train.shape[1], 1)))
        model.add(MaxPooling1D(pool_size=params['pool_size'],
                               strides=None, padding='valid'))
        model.add(Flatten())
        model.add(Dense(params['layer0'], activation=act))
    else:
        model.add(Dense(params['layer0'], activation=act,
                        input_shape=(x_train.shape[1],)))
        model.add(Dropout(dropout))


    # note the change here relative to the regresion problem
    model.add(Dense(params['layer1'], activation=act))
    model.add(Dropout(dropout))

    model.add(Dense(params['layer2'], activation=act))
    model.add(Dropout(dropout))

    model.add(Dense(y_train.shape[1], activation='softmax'))

    model.compile(loss=params['loss_function'], optimizer=optimizer,
                metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                      batch_size=params['batch_size'],
                      epochs=params['epochs'],
                      validation_data=[x_val, y_val],
                      verbose=0,
                      shuffle=True)

    return history, model

In [ ]:
params = {
    'layer0': [9030],
    'layer1':[60], 
    'layer2':[30],
    'dropout': [0.1],
    'activation_function': 'relu',
    'optimizer': ['adam'],
    'cnn': [True],
    'kernel': [8],
    'n_filters': [10],
    'strides': [1],
    'pool_size': [2],
    'loss_function': ['categorical_crossentropy'],
    'batch_size': [32],
    'epochs': [30]
}

In [ ]:
data_by_pair = {pair:[] for pair in target_elements_groups}
for pair in target_elements_groups:
    file_target = storage_directory+'/{}_{}_XY.json'.format(pair[0],pair[1])
    with open(file_target,'r') as f:
        data_by_pair[pair] = [json.loads(line) for line in f.readlines()]

data_by_pair = {pair:[] for pair in target_elements_groups}

models_by_pair = {pair:None for pair in target_elements_groups}
x_valid_by_pair = {pair:None for pair in target_elements_groups}
y_valid_by_pair = {pair:None for pair in target_elements_groups}

for pair in tqdm_notebook(target_elements_groups[0:1]):
    target_file = storage_directory + '/{}_{}_XY.json'.format(pair[0],pair[1])
    with open(target_file, 'r') as f:
        for line in f.readlines():
            cur_data = json.loads(line)
            if cur_data.get('one_hot_coord'):
                data_by_pair[pair].append(cur_data)
    
    X = []
    Y = []    
    for point in data_by_pair[pair]:
        X.append(np.array(point['mu']).reshape(100))
        Y.append(np.array(point['one_hot_coord']))

    X = np.array(X)
    Y = np.array(Y)


    
    x_train, x_test, y_train, y_test = \
      train_test_split(X, Y, test_size=0.1,
                   random_state=42)
    x_train, x_valid, y_train, y_valid = \
      train_test_split(x_train, y_train, test_size=0.1,
                       random_state=42)
    
    
    train_feature_mean = np.mean(np.mean(x_train, axis=0))
    train_feature_std = np.mean(np.std(x_train, axis=0))

    valid_feature_mean = np.mean(np.mean(x_valid, axis=0))
    valid_feature_std = np.mean(np.std(x_valid, axis=0))

    test_feature_mean = np.mean(np.mean(x_test, axis=0))
    test_feature_std = np.mean(np.std(x_test, axis=0))


    print("~~~~~~~~~~~~ Mean +/- Stdev before ~~~~~~~~~~~~~~")
    print("-------------------------------------------------")
    print("Train features %.03f +/- %.03f"
          % (train_feature_mean, train_feature_std))
    print("Valid features %.03f +/- %.03f"
          % (valid_feature_mean, valid_feature_std))
    print("Test features  %.03f +/- %.03f"
          % (test_feature_mean, test_feature_std))


    # Generate a feature and target scaler
    feature_scaler = StandardScaler().fit(x_train)

    # Utilize that scaler on the datasets

    x_train = feature_scaler.transform(x_train)
    x_valid = feature_scaler.transform(x_valid)
    x_test = feature_scaler.transform(x_test)

    train_feature_mean = np.mean(np.mean(x_train, axis=0))
    train_feature_std = np.mean(np.std(x_train, axis=0))

    valid_feature_mean = np.mean(np.mean(x_valid, axis=0))
    valid_feature_std = np.mean(np.std(x_valid, axis=0))

    test_feature_mean = np.mean(np.mean(x_test, axis=0))
    test_feature_std = np.mean(np.std(x_test, axis=0))


    print("~~~~~~~~~~~~~ Mean +/- Stdev after ~~~~~~~~~~~~~~")
    print("-------------------------------------------------")
    print("Train features %.03f +/- %.03f"
          % (train_feature_mean, train_feature_std))
    print("Valid features %.03f +/- %.03f"
          % (valid_feature_mean, valid_feature_std))
    print("Test features  %.03f +/- %.03f"
          % (test_feature_mean, test_feature_std))
    
        
    _, the_model =  classification_model(x_train, y_train, x_valid, y_valid, params)
    
    
    models_by_pair[pair] = the_model
    x_valid_by_pair[pair] = x_valid
    y_valid_by_pair[pair] = y_valid
    
    y_attempt= [np.argmax(y) for y in the_model.predict(x_valid)]
    
